In [13]:
import pandas as pd
import numpy as np
data = pd.DataFrame([['high', 'medium', 'no', 'no'],
                    ['high', 'medium', 'yes', 'no'],
                    ['high', 'high', 'no', 'yes'],
                    ['high', 'high', 'no', 'yes'],
                    ['low', 'high', 'no', 'no'],
                    ['medium', 'medium', 'yes', 'yes'],
                    ['medium', 'high', 'yes', 'no']],
                   index=['sunny', 'sunny','cloudy', 'rainy', 'rainy', 'sunny', 'cloudy'],
                   columns=['temperature', 'humidity', 'windy?', 'Basketball?'])

In [14]:
data.index.name='weather'

In [15]:
data

,temperature,humidity,windy?,Basketball?
weather,,,,
sunny,high,medium,no,no
sunny,high,medium,yes,no
cloudy,high,high,no,yes
rainy,high,high,no,yes
rainy,low,high,no,no
sunny,medium,medium,yes,yes
cloudy,medium,high,yes,no


In [16]:
def entropy(p):
    '''计算熵
    p: a list of probility'''
    entro = 0
    for i in list(p):
        entro -= i * np.log2(i)
    return entro

In [79]:
entropy([])

0

In [23]:
def Gain(par_e, son_p, son_e):
    '''计算信息增益：父节点的熵 - 子节点的归一化熵
    par_e, son_p, son_e
    父节点的熵，子节点的概率, 子节点的对应熵'''
    son_sum = 0
    for sp, se in zip(son_p, son_e):
        son_sum += sp * se
    return par_e - son_sum    

In [24]:
Gain(entropy([3/7, 4/7]), [3/7, 2/7, 2/7], [entropy([1/3, 2/3]), entropy([0.5, 0.5]), entropy([0.5, 0.5])])

0.020244207153756077

In [57]:
def max_Gain(index, par_e, index_gain_cal):
    '''返回最大Gain的index
    index: 属性指标的list
    index_gain_cal: list, 计算对应属性的gain的message'''
    index_gain = []
    for gain in index_gain_cal:
        son_p, son_e = gain   # list的unpack用直接= 就可 a, b = [1, 2], 和zip不同
        index_gain = index_gain + [Gain(par_e, son_p, son_e)]
    index_gain = np.array(index_gain)
    print(index_gain)
    sort = index_gain.argsort()  # 得到的是从小到大排序后的元素的原来的索引值
    print(sort)
    return index[sort[-1]]

In [83]:
index = ['weather', 'temperature', 'humidity', 'windy?']
weather_cal = [[3/7, 2/7, 2/7], [entropy([1/3, 2/3]), entropy([0.5, 0.5]), entropy([0.5, 0.5])]]
temp_cal = [[4/7, 1/7, 2/7], [entropy([1/2, 1/2]), entropy([1]), entropy([1/2, 1/2])]]
hum_cal = [[3/7, 4/7], [entropy([1/3, 2/3]), entropy([1/2, 1/2])]]
windy_cal = [[3/7, 4/7], [entropy([1/3, 2/3]), entropy([1/2, 1/2])]]
par_e = entropy([3/7, 4/7])
cal = [weather_cal, temp_cal, hum_cal, windy_cal]

In [84]:
parent_node = max_Gain(index, par_e, cal)

[0.02024421 0.12808528 0.02024421 0.02024421]
[0 2 3 1]


In [85]:
high_temp_p = [1/2, 1/2]
par_e2 = entropy(high_temp_p)
index2 = index[:]
index2.remove('temperature')  # 。remove在原list操作，返回None
print(index2)
high_temp_weather = [[2/4, 1/4, 1/4], [entropy([1]), entropy([1]), entropy([1])]]
high_temp_hum = [[1/2, 1/2], [entropy([1]), entropy([1])]]
high_temp_windy = [[3/4, 1/4], [entropy([1/3, 2/3]), entropy([1])]]
cal_high_temp = [high_temp_weather, high_temp_hum, high_temp_windy]

['weather', 'humidity', 'windy?']


In [88]:
level1_first_node = max_Gain(index2, par_e2, cal_high_temp)  # 在根据 temperature分类后得到的结果，重新计算打球概率，再计算
print('The first node in 1 level is {}'.format(level1_first_node))

[1.         1.         0.31127812]
[2 0 1]
The first node in 1 level is humidity


# CART 分类树

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
# 准备数据集
iris = load_iris()
# 获取特征和分类标识
features = iris.data
labels = iris.target
# 随机抽取作为测试集和训练集
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.33)
# 创建CART决策树
clf = DecisionTreeClassifier(criterion='gini')  # 基尼系数作为划分标准
# 拟合分类器
clf = clf.fit(train_features, train_labels)
# 用该分类器做预测
test_predict = clf.predict(test_features)
# 比较预测结果和测试集
score = accuracy_score(test_labels, test_predict)
print('CART 分类树准确率： %.4lf' % score)

CART 分类树准确率： 0.9800


# CART 回归树

In [100]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
# 
boston = load_boston()
print(boston.feature_names)
#


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [101]:
features, prices = boston.data, boston.target
train_features, test_features, train_price, test_price = train_test_split(features, prices, test_size=0.33)
dtr = DecisionTreeRegressor()
dtr = dtr.fit(train_features, train_price)
# 进行预测
price_predict = dtr.predict(test_features)
# 评价预测结果
print("回归树二乘偏差均值：{}".format(mean_squared_error(test_price, price_predict)))
print("回归树绝对值偏差均值： {}".format(mean_absolute_error(test_price, price_predict)))

回归树二乘偏差均值：22.669700598802397
回归树绝对值偏差均值： 2.940119760479042


# 手写数据集分类

In [105]:
from sklearn.datasets import load_digits
digits = load_digits()
features, labels = digits.data, digits.target
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.33)
dtc =  DecisionTreeClassifier()
dtc = dtc.fit(train_features, train_labels)
predict_labels = dtc.predict(test_features)
print("回归树二乘偏差均值： {}".format(mean_squared_error(test_labels, predict_labels)))

回归树二乘偏差均值： 3.414141414141414


# Titanic

In [214]:
import pandas as pd

In [264]:
train_data = pd.read_csv('E:/dplrn/Titanic_Data-master/Titanic_Data-master/train.csv')
test_data = pd.read_csv('E:/dplrn/Titanic_Data-master/Titanic_Data-master/test.csv')

In [265]:
# 查看数据
#print(train_data.info())

In [266]:
#print(train_data.describe())

In [267]:
#print(test_data.info())
train_data.shape[0]

891

In [268]:
#print(train_data.head())

In [269]:
#print(train_data.Age, train_data.Fare)
#print(train_data.isnull().any())
print(train_data.Cabin.isnull().any())
#print(train_data.Cabin.isnull() / len(train_data.Cabin))
print(train_data.Cabin.head())

True
0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object


In [270]:
def has_nan(data, columns):
    '''监测是否有NaN
    columns: 要检测的列名'''
    for c in columns:
        if data[c].isnull().any() == True:
            count = data[c].isnull().sum()   # 统计该列na值数量
            p = count / data.shape[0]     # 统计na值占比   DataFrame基本attr
            print('{} column has {} NaN data, count for {}.'.format(c, count,  p))
        else:
            pass

In [271]:
has_nan(train_data, ['Pclass','Name','Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])

Age column has 177 NaN data, count for 0.19865319865319866.
Cabin column has 687 NaN data, count for 0.7710437710437711.
Embarked column has 2 NaN data, count for 0.002244668911335578.


In [272]:
t = [np.nan, 1, 1, 3, 4, 3, 4, 4]
t = pd.DataFrame(t)
t.sum()
t.info()
t.describe()
t.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       7 non-null      float64
dtypes: float64(1)
memory usage: 192.0 bytes


(8, 1)

In [273]:
# 数据清洗
train_data.Age.fillna(train_data.Age.mean(), inplace=True)
test_data.Age.fillna(train_data.Age.mean(), inplace=True)

In [274]:
# Cabin缺失值很多，无法补齐；
# Embarked 缺失值少
train_data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [275]:
train_data.Embarked.fillna('S', inplace=True)

In [276]:
test_data.Embarked.fillna('S', inplace=True)

In [277]:
test_data.Fare.fillna(test_data.Fare.mean(), inplace=True)

In [278]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

In [279]:
#train_labels

In [280]:
#has_nan(train_labels, 'Survived')

In [281]:
# string处理:矩阵化
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse=False)
tr2 = dvec.transform(train_features.to_dict(orient='record'))
te2 = dvec.transform(test_features.to_dict(orient='record'))
print(dvec.feature_names_)
train_features = dvec.fit_transform(train_features.to_dict(orient='record'))
test_features = dvec.fit_transform(test_features.to_dict(orient='record'))
print(dvec.feature_names_)

E:\software\anaco\lib\site-packages\pandas\core\frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


AttributeError: 'DictVectorizer' object has no attribute 'vocabulary_'

In [260]:
# 建模，训练，预测
from sklearn.tree import DecisionTreeClassifier
# ID3 决策树
clf = DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(train_features, train_labels)
predict_labels = clf.predict(test_features)


In [262]:
# 模型评估, 用训练集
acc = round(clf.score(train_features, train_labels), 6)
print("score 准确率为 {}".format(acc))

score 准确率为 0.982043


In [263]:
# 用k折交叉验证评估模型
import numpy as np
from sklearn.model_selection import cross_val_score
print("cross_val_score 准确率 ：{}".format(np.mean(cross_val_score(clf, train_features, train_labels))))

cross_val_score 准确率 ：0.7721863034335572
